# 如何链接可运行对象

关于[LangChain 表达式语言](/docs/concepts/lcel)的一点是，任何两个可运行对象都可以被“链接”在一起形成一个序列。前一个可运行对象的 `.invoke()` 调用的输出将作为输入传递给下一个可运行对象。这可以通过使用 `.pipe()` 方法来完成。

生成的 [`RunnableSequence`](https://api.js.langchain.com/classes/langchain_core.runnables.RunnableSequence.html) 本身也是一个可运行对象，这意味着它可以像其他任何可运行对象一样被调用、流式传输或进一步链接。以这种方式链接可运行对象的优势包括高效的流式传输（序列将在输出可用时立即流式传输输出），以及使用诸如 [LangSmith](/docs/how_to/debugging) 之类的工具进行调试和追踪。

:::info 前提条件

本指南假定您已熟悉以下概念：

- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [提示模板](/docs/concepts/prompt_templates)
- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)

:::

## pipe 方法

为了展示其工作原理，我们来看一个示例。我们将逐步介绍 LangChain 中的一个常见模式：使用 [提示模板](/docs/concepts/prompt_templates) 将输入格式化，然后将其输入到 [聊天模型](/docs/concepts/chat_models)，最后使用 [输出解析器](/docs/concepts/output_parsers) 将聊天消息输出转换为字符串。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="model"
/>
```

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  langchain @langchain/core
</Npm2Yarn>
```

In [ ]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const model = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [2]:
import { StringOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromTemplate("tell me a joke about {topic}")

const chain = prompt.pipe(model).pipe(new StringOutputParser())

提示词和模型都可以运行，且提示词调用的输出类型与聊天模型的输入类型相同，因此我们可以将它们串联在一起。然后，我们可以像调用其他任何可运行对象一样调用这个序列：

In [3]:
await chain.invoke({ topic: "bears" })

"Here's a bear joke for you:\n\nWhy did the bear dissolve in water?\nBecause it was a polar bear!"

### 类型转换

我们甚至可以将这个链与其他可运行对象结合，创建另一个链。这可能需要使用其他类型的可运行对象进行输入/输出格式转换，具体取决于链组件所需的输入和输出。

例如，假设我们想将生成笑话的链与另一个评估生成的笑话是否有趣的链组合起来。

在这种情况下，我们需要谨慎处理输入到下一个链中的格式。在下面的示例中，链中的字典会被自动解析并转换为 [`RunnableParallel`](/docs/how_to/parallel)，它会并行运行其所有值，并返回包含结果的字典。

这恰好是下一个提示模板所期望的相同格式。以下是其实际运行效果：

In [4]:
import { RunnableLambda } from "@langchain/core/runnables";

const analysisPrompt = ChatPromptTemplate.fromTemplate("is this a funny joke? {joke}")

const composedChain = new RunnableLambda({
  func: async (input: { topic: string }) => {
    const result = await chain.invoke(input);
    return { joke: result };
  }
}).pipe(analysisPrompt).pipe(model).pipe(new StringOutputParser())

await composedChain.invoke({ topic: "bears" })

'Haha, that\'s a clever play on words! Using "polar" to imply the bear dissolved or became polar/polarized when put in water. Not the most hilarious joke ever, but it has a cute, groan-worthy pun that makes it mildly amusing. I appreciate a good pun or wordplay joke.'

函数也将被强制转换为可运行对象，因此您也可以向链中添加自定义逻辑。以下链的结果与之前具有相同的逻辑流程：

In [5]:
import { RunnableSequence } from "@langchain/core/runnables";

const composedChainWithLambda = RunnableSequence.from([
    chain,
    (input) => ({ joke: input }),
    analysisPrompt,
    model,
    new StringOutputParser()
])

await composedChainWithLambda.invoke({ topic: "beets" })

"Haha, that's a cute and punny joke! I like how it plays on the idea of beets blushing or turning red like someone blushing. Food puns can be quite amusing. While not a total knee-slapper, it's a light-hearted, groan-worthy dad joke that would make me chuckle and shake my head. Simple vegetable humor!"

参见上面运行的 LangSmith 追踪 [此处](https://smith.langchain.com/public/ef1bf347-a243-4da6-9be6-54f5d73e6da2/r)

但是，请注意，使用此类函数可能会影响流式传输等操作。详见[此部分](/docs/how_to/functions)。

## 下一步

您现在已经了解了一些将两个可运行对象链接在一起的方法。

要了解更多信息，请查看本部分中关于可运行对象的其他操作指南：[本节](/docs/how_to/#langchain-expression-language-lcel)。